Здесь мы дообучим нашу модель, в модели очень много параметров, поэтому она очень долго обучается :(
К сожалению, это одна из возможных причин, почему генератор получится не очень удачным, так как я обучала всего 2 эпохи...
Так же одной из причин станет то, что всн таки наши данные не идеальны и сложно сделать связный текст, изучая цитаты.

In [3]:
import pandas as pd 

filtered_inf = pd.read_csv('C:/Users/Hp/Downloads/filtered_inf.csv')
quotes = filtered_inf['quote'].tolist()

In [ ]:
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate
#Если что-то не работает, то можно попробовать запустить эту ячейку.

Для обучения модели сначала преобразуем данные в текстовый файл, а потом токенезируем данные.

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
 
train_path = 'train_dataset.txt'
with open(train_path, "w", encoding="utf-8") as f:
    for quote in quotes[2648:]:
        f.write(quote + "\n")
    for quote in quotes:
        f.write(quote + "\n")

train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=128)
  
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned",
    overwrite_output_dir=True, 
    num_train_epochs=2, 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=32,  
    warmup_steps=10,
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-3),None) 
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("model2/") #Сохраним модель в папку model2

Теперь перейдем в файл analysis.